<a href="https://colab.research.google.com/github/aman-kumar-004/Coursera_Capstone/blob/main/W4_IBM_Data_Science_Capstone_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geocoder 

     |████████████████████████████████| 102kB 4.7MB/s 


### Bsuiness Problem

The problem I’m trying to solve is to establish multiple  recycling center in strategic places inside the city so that the maximum volume of the waste gets recycled also providing revenue to the organization at the same time.

In [2]:
#importing necessary libraries 

import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

from geopy.geocoders import Nominatim
import geocoder
import folium

### Data Used

There are two approaches to find the solution for this problem. The first is to target the residential areas as waste is generated in large volume from there also, but due to the limitations in data available for the Bengaluru city, it is not possible. Second to target the areas where there is a maximum number of businesses.

For this first, I have scrapped the Wikipedia page which has a list of all the neighborhoods in the city, and then I have collected the coordinates of each neighborhood using geocoder and put that in a pandas data frame.



In [3]:
#wikipedia page for all the neighborhood in Bengaluru city

url = 'https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore'

html_file  = requests.get(url).text

html_file



'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Category:Neighbourhoods in Bangalore - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"02b58399-229c-4aea-9173-d704346714db","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Neighbourhoods_in_Bangalore","wgTitle":"Neighbourhoods in Bangalore","wgCurRevisionId":796834779,"wgRevisionId":796834779,"wgArticleId":3838632,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Geography of Bangalore","Neighbourhoods in Karnataka"],"wgPageContentLanguage":"en","wgPageCon

In [4]:
#Web Scrapping

soup = BeautifulSoup(html_file)

soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Neighbourhoods in Bangalore - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"02b58399-229c-4aea-9173-d704346714db","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Neighbourhoods_in_Bangalore","wgTitle":"Neighbourhoods in Bangalore","wgCurRevisionId":796834779,"wgRevisionId":796834779,"wgArticleId":3838632,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Geography of Bangalore","Neighbourhoods in Karnataka"],"wgPageContentLanguage":"en","wgPageContentMo

In [5]:
#Cleaning

neigh_list = []
for row in soup.find_all("div", class_="mw-category")[0].findAll('li')[4:]:
  neigh_list.append(row.text)

neigh_list

['Agara, Bangalore',
 'Ananthnagar',
 'Anjanapura',
 'Arekere',
 'Austin Town',
 'Babusapalya',
 'Bagalur, Bangalore Urban',
 'Bahubalinagar',
 'Baiyyappanahalli',
 'Baiyyappanahalli Manavarthe Kaval',
 'Banashankari',
 'Banaswadi',
 'Basavanagudi',
 'Basaveshwaranagar',
 'Bellandur',
 'Bengaluru Pete',
 'Bharathnagar',
 'Bilekahalli',
 'Binnamangala Manavarthe Kaval',
 'Binnamangala, Bangalore',
 'Bommanahalli',
 'Bommasandra',
 'Brookefield',
 'BTM Layout',
 'Byatarayanapura',
 'Carmelaram',
 'Chamrajpet',
 'Chandapura',
 'Chickpet',
 'Chikkalasandra',
 'Cooke Town',
 'Cox Town, Bangalore',
 'CV Raman Nagar',
 'Dasarahalli',
 'Devara Jeevanahalli',
 'Devarachikkanahalli',
 'Dollars Colony',
 'Domlur',
 'Ejipura',
 'Electronic City',
 'Fraser Town, Bangalore',
 'G. M. Palya',
 'Gandhi Bazaar',
 'Gandhi Nagar, Bangalore',
 'Gangamma Circle',
 'Ganganagar, Bangalore',
 'Garudacharpalya',
 'Girinagar',
 'Gottigere',
 'Gowdanapalya',
 'Hanumanthanagar, Bengaluru',
 'Hebbagodi',
 'Hebbal, 

In [6]:
#converting into a pandas dataframe
df = pd.DataFrame(data = neigh_list, columns= ['Neighborhood'])

df.head()

,Neighborhood
0,"Agara, Bangalore"
1,Ananthnagar
2,Anjanapura
3,Arekere
4,Austin Town


In [7]:
df.shape

(139, 1)

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bengaluru, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df['Neighborhood'].tolist() ]

In [10]:
coords

[[12.842830000000049, 77.48759000000007],
 [12.954080000000033, 77.54135000000008],
 [12.858110000000067, 77.55909000000008],
 [12.885680000000036, 77.59668000000005],
 [12.963480000000061, 77.61297000000008],
 [13.027180000000044, 77.65045000000003],
 [13.07728933897749, 77.65760586439058],
 [13.044710000000066, 77.55008000000004],
 [12.98912000000007, 77.65109000000007],
 [12.98912000000007, 77.65109000000007],
 [12.922310000000039, 77.56988000000007],
 [13.019526632499165, 77.65502770568062],
 [12.938980000000072, 77.57137000000006],
 [12.986740000000054, 77.54389000000003],
 [12.927340000000072, 77.67169000000007],
 [12.966180000000065, 77.58690000000007],
 [13.083010000000058, 77.54779000000008],
 [12.900090000000034, 77.60433000000006],
 [13.015990000000045, 77.61046000000005],
 [12.984031733863102, 77.64047977013732],
 [12.903080000000045, 77.62444000000005],
 [12.817530000000033, 77.67879000000005],
 [12.993330000000071, 77.66123000000005],
 [12.914950000000033, 77.610010000000

In [11]:
#temporary dataframe
df_cords = pd.DataFrame(data = coords, columns = ['Latitude', 'Longitude'])

df_cords.head()

,Latitude,Longitude
0,12.84283,77.48759
1,12.95408,77.54135
2,12.85811,77.55909
3,12.88568,77.59668
4,12.96348,77.61297


In [13]:
df['Latitude'] = df_cords['Latitude']
df['Longitude'] = df_cords['Longitude']

df.head()

,Neighborhood,Latitude,Longitude
0,"Agara, Bangalore",12.84283,77.48759
1,Ananthnagar,12.95408,77.54135
2,Anjanapura,12.85811,77.55909
3,Arekere,12.88568,77.59668
4,Austin Town,12.96348,77.61297


In [14]:
#extracting the coordinates of the city

address = 'Bengaluru, India'
geolocator = Nominatim(user_agent="my application")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print("Bengaluru latitude {} and longitude {}".format(latitude, longitude ))

Bengaluru latitude 12.9791198 and longitude 77.5912997


In [15]:
#map of all the neighborhood in Bengaluru city

map = folium.Map(location= [latitude, longitude], zoom_start=12)

for lat, lng, neigh in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
  label = '{}'.format(neigh)
  label = folium.Popup(label, parse_html= True)
  folium.CircleMarker(
       [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
    
  

In [16]:
map

In [17]:
map.save("/content/drive/MyDrive/IBM Data Science/neigh_map")

In [18]:
#Forsquare Credentials

CLIENT_ID = 'IFMREZT4DTZVEC4ZZQS4YNGJ3SAQNWJ4KUP5W2ZUD0AKLFZ5'
CLIENT_SECRET = '2DBHK1MZLROLH1EBZENG25QTF5YVD5B1W1ENU2E4J1XQGPL3'
#removed before uploading in github

VERSION = '20180604'
LIMIT = 50

print("Your Credentials")
print('Client Id: ' + CLIENT_ID)
print('Client Secret: ' + CLIENT_SECRET)

Your Credentials
Client Id: IFMREZT4DTZVEC4ZZQS4YNGJ3SAQNWJ4KUP5W2ZUD0AKLFZ5
Client Secret: 2DBHK1MZLROLH1EBZENG25QTF5YVD5B1W1ENU2E4J1XQGPL3


In [19]:
#To get the list of venues present in each neighborhood

venues_list =[]
radius = 400
for lat, lng, neigh in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
  results = requests.get(url).json()["response"]['groups'][0]['items']
  for venue in results:
    venues_list.append((
    neigh,
    lat, 
    lng, 
    venue['venue']['name']))
  
  
  



In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues_list)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName']

print(venues_df.shape)
venues_df.head(10)

(764, 4)


,Neighborhood,Latitude,Longitude,VenueName
0,Ananthnagar,12.95408,77.54135,Big Bazaar
1,Ananthnagar,12.95408,77.54135,KSRTC Satellite Bus Stand
2,Ananthnagar,12.95408,77.54135,Imperial Restaurant
3,Ananthnagar,12.95408,77.54135,Kadamba Veg
4,Ananthnagar,12.95408,77.54135,Adyar Anand Bhavan
5,Arekere,12.88568,77.59668,Decathlon Sports India Pvt Ltd
6,Arekere,12.88568,77.59668,Benison Super Market
7,Arekere,12.88568,77.59668,Aishwarya Park/land
8,Arekere,12.88568,77.59668,Star Bazaar
9,Arekere,12.88568,77.59668,Triton


In [21]:
venues_df.count()

Neighborhood    764
Latitude        764
Longitude       764
VenueName       764
dtype: int64

In [22]:
x = venues_df.groupby(by='Neighborhood').count()
x = x.sort_values(by='VenueName', ascending=False)

x.head()

,Latitude,Longitude,VenueName
Neighborhood,,,
Milk Colony,50,50,50
"Jayanagar, Bangalore",41,41,41
"Fraser Town, Bangalore",33,33,33
BTM Layout,29,29,29
Brookefield,28,28,28


In [23]:
x = x.reset_index()
x = x.drop(columns=['Latitude','Longitude'])
x.head()

,Neighborhood,VenueName
0,Milk Colony,50
1,"Jayanagar, Bangalore",41
2,"Fraser Town, Bangalore",33
3,BTM Layout,29
4,Brookefield,28


In [24]:
cordinates = [get_latlng(neigh) for neigh in x['Neighborhood'].to_list() ]

In [25]:
cordinates

[[13.009660000000054, 77.55683000000005],
 [12.928720000000055, 77.58281000000005],
 [12.998940000000061, 77.61276000000004],
 [12.914950000000033, 77.61001000000005],
 [12.993330000000071, 77.66123000000005],
 [12.976750000000038, 77.59879000000006],
 [12.944780000000037, 77.57213000000007],
 [12.912220000000048, 77.64470000000006],
 [12.927340000000072, 77.67169000000007],
 [12.962330000000065, 77.60123000000004],
 [12.990730000000042, 77.58861000000007],
 [12.984031733863102, 77.64047977013732],
 [12.938980000000072, 77.57137000000006],
 [12.885680000000036, 77.59668000000005],
 [13.024450000000058, 77.59590000000003],
 [13.029550000000029, 77.54022000000003],
 [12.95613000000003, 77.73196000000007],
 [12.909950000000038, 77.55051000000003],
 [12.966200000000072, 77.64982000000003],
 [12.967520000000036, 77.71500000000003],
 [13.014300000000048, 77.63685000000004],
 [12.955650000000048, 77.65335000000005],
 [12.922450000000026, 77.61033000000003],
 [13.062710000000038, 77.5855000000

In [26]:
df_cordinates = pd.DataFrame(data = cordinates, columns= ['latitude', 'longitude'])
df_cordinates.head()

,latitude,longitude
0,13.00966,77.55683
1,12.92872,77.58281
2,12.99894,77.61276
3,12.91495,77.61001
4,12.99333,77.66123


In [27]:
x['Latitude'] = df_cordinates['latitude']
x['Longitude'] = df_cordinates['longitude']
x.head()

,Neighborhood,VenueName,Latitude,Longitude
0,Milk Colony,50,13.00966,77.55683
1,"Jayanagar, Bangalore",41,12.92872,77.58281
2,"Fraser Town, Bangalore",33,12.99894,77.61276
3,BTM Layout,29,12.91495,77.61001
4,Brookefield,28,12.99333,77.66123


In [28]:
x.columns = ['Neighborhood','Number of Venues','Latitude','Longitude']
x.head()

,Neighborhood,Number of Venues,Latitude,Longitude
0,Milk Colony,50,13.00966,77.55683
1,"Jayanagar, Bangalore",41,12.92872,77.58281
2,"Fraser Town, Bangalore",33,12.99894,77.61276
3,BTM Layout,29,12.91495,77.61001
4,Brookefield,28,12.99333,77.66123


In [29]:
recycle_centre = x[x['Number of Venues'] > 10 ]
recycle_centre.shape

(19, 4)

## Result

In [30]:
recycle_centre

,Neighborhood,Number of Venues,Latitude,Longitude
0,Milk Colony,50,13.009660,77.55683
1,"Jayanagar, Bangalore",41,12.928720,77.58281
2,"Fraser Town, Bangalore",33,12.998940,77.61276
3,BTM Layout,29,12.914950,77.61001
4,Brookefield,28,12.993330,77.66123
5,"Statue of Queen Victoria, Bangalore",19,12.976750,77.59879
6,Gandhi Bazaar,18,12.944780,77.57213
7,HSR Layout,17,12.912220,77.64470
8,Bellandur,16,12.927340,77.67169
9,Richmond Town,15,12.962330,77.60123


In [32]:
recycle_centre.to_csv("/content/drive/MyDrive/IBM Data Science/recycle_center.csv")

In [33]:
#Map of the recycling center in city

map_rec_centre = folium.Map(location= [latitude, longitude], zoom_start=12)

for lat, lng, neigh in zip(recycle_centre['Latitude'], recycle_centre['Longitude'], recycle_centre['Neighborhood']):
  label = '{}'.format(neigh)
  label = folium.Popup(label, parse_html= True)
  folium.CircleMarker(
       [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_rec_centre)  
    

In [34]:
map_rec_centre

In [37]:
map_rec_centre.save("/content/drive/MyDrive/IBM Data Science/rec_cent_map.jpeg")

In [36]:
'''
Due to the limmitatioins of api calls, I have to limit the number of venues per neighborhood. This is to showing 
my concept of opening recycling centers based on the number of venues present in each neighborhood so that recycling
happens in a greater efficiency.
'''

'\nDue to the limmitatioins of api calls, I have to limit the number of venues per neighborhood. This is to showing \nmy concept of opening recycling centers based on the number of venues present in each neighborhood so that recycling\nhappens in a greater efficiency.\n'